#### Importing libraries 

In [1]:
import cv2
import pytesseract
import numpy as np 
import pandas as pd
import re
import gradio as gr
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans

## Machine Learning Engines - Syntethic training for zero - dataset Logic

#### The model of classification by identfying the move of a player. 
#### The regressor identifies and examines based on evaluation score. 
#### Clustering identifying the elo and playstyle of the player. 

In [2]:
# Classification: 2=Good, 1=Mistake, 0=Blunder
clf = RandomForestClassifier().fit([[0.1], [0.8], [2.1]], [2, 1, 0]) 
# Regression: Predicts Win % based on Eval Score
reg = RandomForestRegressor().fit([[1.0], [0.0], [-1.0]], [80, 50, 20]) 
# Clustering: Profiles Playstyle (Mean Eval vs Volatility)
km = KMeans(n_clusters=2, n_init=10).fit([[0.5, 0.2], [1.5, 1.8]])

## Image and data processing 

#### Image proecessing in synthetic format and zero dataset.

In [3]:
#Analyzing image
def analyze_game(image): 
    if image is None:
        return "No image provided", "0%", "N/A", "N/A", None, None

#### In order to upload your screenshot, you just have to upload the screenshot of the game. As the machine will identify your moves. 

In [4]:
def analyze_game(image): 
    if image is None:
        return "No image provided", "0%", "N/A", "N/A", None, None
    
    try:
        #Analyzing computer vision
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
        
        # Adaptive thresholding.
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
        
        #Extraction of OCR.
        text = pytesseract.image_to_string(thresh)
        nums = [float(n) for n in re.findall(r"[-+]?\d*\.\d+|\d+", text)]
        
        if len(nums) < 2:
            return "Insufficient Data", "0%", "N/A", "N/A", None, thresh

        #Machine learning - Identifying pipeline. 
        df = pd.DataFrame({"Move": range(len(nums)), "Score": nums})
        df['Delta'] = df['Score'].diff().abs().fillna(0)

        # Classification & Regression
        labels = {2: "✅ Good", 1: "⚠️ Mistake", 0: "🚨 Blunder"} #Identifying the accuracy.
        df['Quality'] = [labels[q] for q in clf.predict(df[['Delta']].values)]
        df['WinProb'] = reg.predict(df[['Score']].values)

        # Clustering (Playstyle Analysis)
        style_idx = km.predict([[np.mean(nums), np.std(nums)]])[0]
        styles = ["Strategic/Solid", "Aggressive/Tactical"] 
        playstyle = styles[style_idx]

        # Ranking (Performance Tier)
        accuracy_val = max(0, 100 - (df['Delta'].mean() * 10))
        rank = "Elite" if accuracy_val > 85 else "Standard" if accuracy_val > 65 else "Developing"

        # Pattern Insight
        pattern = f"Frequent Error: {df['Quality'].mode()[0]}" if not df.empty else "N/A"

        #Visualization and analysis. 
        fig = px.line(df, x="Move", y="Score", title="Game Performance Trend", markers=True)
        fig.add_hline(y=0, line_dash="dash", line_color="gray")
        fig.update_layout(template="plotly_dark")
        
        return rank, f"{accuracy_val:.1f}%", playstyle, pattern, fig, thresh

    except Exception as e:
        return f"Error: {str(e)}", "N/A", "N/A", "N/A", None, None

### Gradio Interface 


In [5]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# ♟️ Chess Game Analysis Machine")
    gr.Markdown("Upload a screenshot of your game evaluation to see ML insights.")
    
    with gr.Row():
        with gr.Column():
            input_img = gr.Image(label="Game Screenshot", type="numpy")
            btn = gr.Button("Analyze Data", variant="primary")
        with gr.Column():
            with gr.Row():
                rank_out = gr.Textbox(label="Skill Rank")
                acc_out = gr.Textbox(label="Accuracy %")
            style_out = gr.Textbox(label="Playstyle (Clustering)")
            assoc_out = gr.Textbox(label="Pattern Insight (Association)")

    with gr.Tabs():
        with gr.TabItem("📊 Performance Plot"):
            plot_out = gr.Plot()
        with gr.TabItem("👁️ Vision Debug"):
            vision_out = gr.Image(label="OCR Binary View")

    btn.click(analyze_game, input_img, 
              [rank_out, acc_out, style_out, assoc_out, plot_out, vision_out])

if __name__ == "__main__":
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://9c32cf97fcbe701794.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
